In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd

tf.set_random_seed(1)
np.random.seed(1)

/Users/Sylvain/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [34]:
eval_csv_path = os.path.join('..', 'data/DAGM 2007 - Splitted', '6', 'test_files.csv')
df = pd.read_csv(train_csv_path, sep=",")
df.head()

,Unnamed: 0,original_path,is_healthy,target
0,0,data\957.png,1,test\001.png
1,1,def\87.png,-1,test\002.png
2,2,data\180.png,1,test\003.png
3,3,def\122.png,-1,test\004.png
4,4,data\12.png,1,test\005.png


In [43]:
def csv_generator(csv_path, class_nbr, keep_fn=False):
    df = pd.read_csv(csv_path, sep=",")
    for index, row in df.iterrows():
        img_path = os.path.join('..', 'data/DAGM 2007 - Splitted', str(class_nbr), row['target'].replace("\\", "/"))
        img_fn = os.path.basename(img_path)
        img_encoded = np.load("../tmp/cnn_output/VGG16/{}/{}.npy".format(
            class_nbr,
            row['target'].replace("\\", "/")
        ))
        label = row['is_healthy']
        if keep_fn:
            yield (img_fn, img_encoded, label)
        yield (img_encoded, label)

def eval_input_fn(eval_csv_path=eval_csv_path, target_width=224, batch_size=2):
    dataset = tf.data.Dataset.from_generator(
        generator=lambda: csv_generator(eval_csv_path, 6),
        output_types=(tf.float32, tf.int32)
    )
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()

eval_input_fn(eval_csv_path)

(<tf.Tensor 'IteratorGetNext_7:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'IteratorGetNext_7:1' shape=<unknown> dtype=int32>)

In [44]:
sess = tf.Session()
sess.run(eval_input_fn(eval_csv_path))

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([ 1, -1], dtype=int32))